# Multiple linear regression

In this notebook, we will see how to do multiple linear regression in Python using statsmodels and scikit-learn.

First we import the standard packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
#from scipy import stats

import statsmodels.api as sm
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

As example data, we will use the Ames housing data, see kaggle: https://www.kaggle.com/datasets/marcopale/housing?resource=download. It is also available on Moodle.

In [ ]:
ames = pd.read_csv("AmesHousing.csv")

In [ ]:
ames

In [ ]:
ames.info()

As we now have more than one predictor/feature/independent variable, we cannot visualize the linear relationship anymore. thus we will skip the plotting.

Now, however, we have a more complex dataset and we need to do a bit of preprocessing before we can use it for multiple linear regression.

For simplicity we will only use the following predictor variable: 

- `Lot Area` (the size of the lot in square feet)
- `Overall Cond` (rating of the overall condition of the house)
- `Year Built` (Original construction year)
- `Gr Liv Area` (Above grade (ground) living area in square feet)
- `TotRms AbvGrd`(Total rooms above grade, excluding bathrooms)
- `Mo Sold` (Month Sold)
- `Yr Sold` (Year Sold)

In [ ]:
X_ames = ames[["Lot Area", "Overall Cond", "Year Built", "Gr Liv Area", "TotRms AbvGrd", "Mo Sold", "Yr Sold"]]
X_ames

For the response/target/dependent variable, we use `SalePrice`

In [ ]:
y = ames["SalePrice"]

## Fitting a multiple linear regression model using OLS and statsmodels

We can fit a multiple regression model to our data using OLS just as for simple linear regression.

We add an intercept to X as in the case of simple linear regression

In [ ]:
X_ames_wInt = sm.add_constant(X_ames)

In [ ]:
X_ames_wInt

We can now with a multiple linear regression model using OLS just as before

In [ ]:
mulinreg_model = sm.OLS(y, X_ames_wInt).fit()

We can get a lot of information about our model from the summary method.

In [ ]:
mulinreg_model.summary()

**Interpretation of the model**

We see a fairly high R-squared of 0.688.

The intercept, month sold, and year sold are not statistically significantly different from 0, the rest of the predictor variables are. Depending on the purpose of our modeling, it might make sense to remove the variables which coefficients are not significantly different from zero (more on this later).

### Retrieving coefficients and plotting fitted regression line

We can also get the parameters or coefficients from the fitted model the following way:

In [ ]:
mulinreg_model.params

We can get the predictions as before using the predict method

In [ ]:
pred_y = mulinreg_model.predict()
pred_y

### Model performance measures

We can also get the R-squared directly from the fitted model object.

In [ ]:
mulinreg_model.rsquared

In [ ]:
mulinreg_model.rsquared_adj

We can get the residuals of the model also

In [ ]:
mulinreg_model.resid

Using these, we can calculate the *MAE* (Mean Absolute Error).

In [ ]:
np.mean(np.abs(mulinreg_model.resid))

and the *MSE* (Mean Squared Error).

In [ ]:
np.mean(mulinreg_model.resid**2)

and the *RMSE* (Root Mean Squared Error).

In [ ]:
np.sqrt(np.mean(mulinreg_model.resid**2))

## Fitting multiple linear regression using scikit-learn

We can also use the scikit-learn package to fit a multiple regression model.

Then we define a linear regression mode.

In [ ]:
mulinreg_model_sk = linear_model.LinearRegression()

We can now fit the model.

In [ ]:
mulinreg_model_sk.fit(X_ames, y)

Getting the coefficient in scikit-learn will give us all the coefficient except the coefficient for the intercept. That we have to get seperately.

In [ ]:
mulinreg_model_sk.coef_

In [ ]:
mulinreg_model_sk.intercept_

We see that we get the same value as for statsmodels OLS.

To get the prediction of the model, we can use the `.predict` method again.

In [ ]:
mulinreg_model_sk.predict(X_ames)

We can use the metrics submodule from scikit-learn again to calculate the performance of the multiple linear regression model.

All of these take the true and the predicted values as arrays. Thus, we first define a variable for the predicted.

In [ ]:
y_pred_sk = mulinreg_model_sk.predict(X_ames)

In [ ]:
r2_score(y, y_pred_sk)

In [ ]:
mean_absolute_error(y, y_pred_sk)

In [ ]:
mean_squared_error(y, y_pred_sk)

In [ ]:
root_mean_squared_error(y, y_pred_sk)

In [ ]:
root_mean_squared_error(y, pred_y)

## Dealing with categorical variables

We will use the categorical variable `Bldg Type` (Type of dwelling) from the ames dataset for these examples.

In [ ]:
ames[["Bldg Type"]].groupby(ames["Bldg Type"]).count()

In [ ]:
pd.get_dummies(ames["Bldg Type"], drop_first=True, dtype = "int")

We use the `get_dummies`from pandas.

In [ ]:
X_ames_wInt = X_ames_wInt.join(pd.get_dummies(ames["Bldg Type"], drop_first=True, dtype = "int"))

X_ames = X_ames.join(pd.get_dummies(ames["Bldg Type"], drop_first=True, dtype = "int"))

In [ ]:
X_ames_wInt

In [ ]:
X_ames

Note that it is the *1Fam* building type that is left out, thus it will be the reference type used in interpreting the regression coefficent for this categorical variable.

We can now retrain our multiple regression models

In [ ]:
mulinreg_model = sm.OLS(y, X_ames_wInt).fit()

In [ ]:
mulinreg_model.summary()

In [ ]:
mulinreg_model.params["2fmCon"]

**Interpretation of the model**

We can see that we increased the R-squared of the model, and several of the building types are significant. As we are dealing with multiple regression, we should actually look at adjusted R-squared instead of R-squared. But Adjusted R-squared has also improved.

That the coefficient for 2fmCon is -12976 (-1.298e+04) means that the selling price for a 2fmCon building is 12976 less than a 1Fam building, everying else being equal.

Calculating performance metrics.

In [ ]:
pred_y = mulinreg_model.predict()
pred_y

In [ ]:
r2_score(y, pred_y)

In [ ]:
root_mean_squared_error(y, pred_y)

We can see that we do get a better performance.

Training the model with scikit-learn

In [ ]:
mulinreg_model_sk = linear_model.LinearRegression()

In [ ]:
mulinreg_model_sk.fit(X_ames, y)

In [ ]:
y_pred_sk = mulinreg_model_sk.predict(X_ames)

In [ ]:
r2_score(y, y_pred_sk)

In [ ]:
root_mean_squared_error(y, y_pred_sk)

## Looking at assumptions and problems

We will now look at the assumptions and potential problems for our multiple linear regression model.

If these assumptions are not meet, it means that we cannot completely trust the statistical calculations such as the p-values. Moreover, our evaluation metrics might not truly represent the performance of our model.

### Non-linearity of the data

One of the assumptions of linear regression is that there is a linear relationship between the independent (X) variables and the dependent variable (y). 

For simple linear regression, we can make a scatterplot of the x and y variable and visually inspect for linear relationship. However, for multiple linear regression, we cannot do that. Instead, we can plot the residuals versus the predicted values:

In [ ]:
sns.scatterplot(x = pred_y, y = mulinreg_model.resid)
plt.plot(pred_y, np.repeat(0, len(pred_y)), color = "orange")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.show()

They look like they go up a bit, maybe the relationship is not quite linear, but the points almost fall equally along the x-axis. In other words, there is no clear pattern suggesting that the data is not approximately linear.

### Correlation of error terms 

Another assumption of linear regression is that the error terms are uncorrelated, in other words, the i'th error term $e_i$ does not tell us anything about i+1'th error term $e_{i+1}$. Thus, to investigate this, we can plot the residuals in order of their appearance (or by time, if there is a time variable). As we have no time variable, we can instead plot the residuals versus their row number.

In [ ]:
plt.rc("figure", figsize=(12, 8))
sns.lineplot(x = range(0,len(mulinreg_model.resid)), y = mulinreg_model.resid)
plt.ylabel("Residuals")
plt.xlabel("Data point index")
plt.show()

It is actually a bit hard to see from this plot if there is any correlation between the error terms, in terms of longer sections of value above or below 0. However, there are no clear such sections or pattern, so there is no obvious correlation, at least. Thinking about the data, it is also not obvious how a correlation among errors should have arisen.

### Non-constant variance of error terms

Another assumption of linear regression is that the error terms have constant variance. One common way of seeing non-constant error terms it to look at the plot of residuals versus predicted values a look whether the variance increase as the predicted values do - it will look like a funnel. So let us look at this plot again:

In [ ]:
plt.rc("figure", figsize=(8, 5))
sns.scatterplot(x = pred_y, y = mulinreg_model.resid)
plt.plot(pred_y, np.repeat(0, len(pred_y)), color = "orange")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.show()

It indeed looks like there is a bit of a funnel, that is, an increase in variance. Thus, the assumption of constant variance of the error terms might indeed be violated.

 One possible solution would be to transform the dependent/target variable $y$ to $log(y)$ instead. This will potentially work in this case, but it will also make the interpretation of the model harder (the coefficient will mean something else) and we need to transform each prediction as well to get back the price of the house. We will not do this not, however.

### Outliers

Outliers in regard to linear regression models, are points for which the predicted value is very far from the actual values. These we can also spot in the residual vs predicted values plot. So let us not make the plot again, but just look at the plot above.

The three points, at the bottom right, looks like they could be outliers as their absolute residual values are quite high. Thus, it would make sense to investigate those point further to see if there are any errors in the data or other reasons from them being outliers.

### High leverage points

High leverage points are a bit like residuals. They are point that as a high influence on how the model looks. In simple linear regression, this means that a high leverage point might heavily affect where the regression line lies. Note that outliers are extreme values in a sense, but they do not need to have high leverage as they might not affect the actual model fit that much.

High leverage points are usually points that have an x-value far away from the other x-values in the dataset. This is easy to spot in simple linear regression where we only have on x variable. However, it is much harder to spot directly for multiple linear regression. Luckily, there is a leverage statistics we can calculate for each point based on a linear regression model, which we can use to make a "Leverage plot".

We can calculate the leverage statistics for each point in the following for statsmodels (does not work for scikit-learn models):

In [ ]:
leverageStats = mulinreg_model.get_influence().hat_matrix_diag
leverageStats

In [ ]:
plt.rc("figure", figsize=(12, 8))
sns.scatterplot(x = range(0, len(leverageStats)), y = leverageStats)
plt.ylabel("Leverage")
plt.xlabel("Data point index")
plt.show()

Here there are clearly 3(/4) point that stands out with much higher leverage than the other points. Thus, it could again be useful to investigate these points further. (We will not do that now.)

### Collinearity

Collinearity can also be a problem for linear regression models and refers to the existence of high correlation between two or more of the independent/predictor variables. IF two predictor variables are highly correlated, it can be hard/impossible for a linear regression model to separate out the effect on the response variable, coming from each of them - one might have a really high positive effect, while the other might have a really high negative effect, but in reality neither of them might have a big impact on the response variable.

The easiest way to spot collinearity between any pair of variables is to look at the correlation matrix:

In [ ]:
X_ames.corr()

We do not see any large correlation between the x variables here that could be a problem. The only correlation that is a bit (to?!) high is the one between `TotRms AbvGrd` and `Gr Liv Area`. The easiest way to deal with collinearity among the predictor variables is simply to drop one of them. Usually this well not make the model much worse as on the of the variable contains most of the information contained in the other variable.

It is worth noticing that we could potentially have collinearity between a set of more than two variables. this could create problems for the linear regression model, but it is not always possible to spot this from the pairwise correlation matrix.

## Improving the model

As we mentioned previously, it might sometimes make sense to remove variables whose coefficient is not significantly different from zero. It does not make sense to remove some of the dummy variables created from the same variable, so we would not remove `TwnhsE` even though it has a very high p-value. However, we might consider removing the intercept (`const`), `Mo Sold`, and `Yr Sold`.

If we just try to achieve the highest predictive accuracy (RMSE for instance), it makes good sense to try without those variables. If are interested in answering the more inferential question of what affect the sales price of a house, it also makes sense to remove the non-significant variables. However, in some cases we might want to keep the variable if we are interested in those particular, that is if we are interested in the effect of the sales data on the price of the house. 

Let us say we are interested in predictive performance (measured by RMSE) and in achieving a high adjusted R-square, and let see if we can improve those by removing non-significant variables

In [ ]:
X_ames_onlySig = X_ames_wInt.drop(columns=["const", "Mo Sold", "Yr Sold"])

In [ ]:
mulinreg_model_onlySig = sm.OLS(y, X_ames_onlySig).fit()

In [ ]:
mulinreg_model_onlySig.summary()

Now we clearly improved the adjusted R-square from $0.703$ to $0.928$, which is quite of an improvement! Let us see if we also improved RMSE:

In [ ]:
pred_y_onlySig = mulinreg_model_onlySig.predict()
root_mean_squared_error(y, pred_y_onlySig)

Now, for predictive performance the model actually got worse, in the sense that RMSE increased substantially from $43473$ to $53070$. Leaving out variables never improve predictive performance (measured by RMSE) for linear regression when we measure RMSE on the same data as we trained on. However, as we will talk about next time, we should never measure predictive performance on the same data as we trained the model. If we measure RMSE on data the model has not seen before, we might improve performance be removing some insignificant variables - this is especially true for other model types than linear regression.